In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [5]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         rotation_range=15,
                                         height_shift_range=0.2,
                                         width_shift_range=0.2,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [9]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predict = Dense(10, activation='softmax')(x)

vgg16_model = Model(inputs=vgg16_base_model.input, outputs=predict)

In [10]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_2 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 flatten_2 True
20 dense_4 True
21 batch_normalization_3 True
22 dropout_3 True
23 dense_5 True
24 batch_normalization_4 True
25 dropout_4 True
26 dense_6 True


In [11]:
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])


vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

vgg16_model.save('vgg16_model_fc.h5')

Epoch 1/15
156/156 [==============================] - 330s 2s/step - loss: 2.2529 - acc: 0.3022 - val_loss: 1.3992 - val_acc: 0.4670
Epoch 2/15
156/156 [==============================] - 326s 2s/step - loss: 1.1133 - acc: 0.6101 - val_loss: 1.0798 - val_acc: 0.6355
Epoch 3/15
156/156 [==============================] - 323s 2s/step - loss: 0.7546 - acc: 0.7477 - val_loss: 0.9663 - val_acc: 0.6747
Epoch 4/15
156/156 [==============================] - 322s 2s/step - loss: 0.6134 - acc: 0.7941 - val_loss: 1.2106 - val_acc: 0.6429
Epoch 5/15
156/156 [==============================] - 321s 2s/step - loss: 0.5354 - acc: 0.8229 - val_loss: 1.0181 - val_acc: 0.6794
Epoch 6/15
156/156 [==============================] - 317s 2s/step - loss: 0.4713 - acc: 0.8402 - val_loss: 0.9991 - val_acc: 0.6764


In [13]:
vgg16_model = load_model('vgg16_model_fc.h5')

for layer in vgg16_model.layers[:15]:
    layer.trainable = False
    
for layer in vgg16_model.layers[15:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=5e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])

vgg16_model.save('vgg16_model_15.h5')

Epoch 1/20
156/156 [==============================] - 324s 2s/step - loss: 0.4192 - acc: 0.8705 - val_loss: 0.6258 - val_acc: 0.8190
Epoch 2/20
156/156 [==============================] - 328s 2s/step - loss: 0.1379 - acc: 0.9596 - val_loss: 0.8048 - val_acc: 0.8193
Epoch 3/20
156/156 [==============================] - 324s 2s/step - loss: 0.0893 - acc: 0.9755 - val_loss: 0.5617 - val_acc: 0.8524
Epoch 4/20
156/156 [==============================] - 321s 2s/step - loss: 0.0758 - acc: 0.9802 - val_loss: 0.5994 - val_acc: 0.8306
Epoch 5/20
156/156 [==============================] - 320s 2s/step - loss: 0.0610 - acc: 0.9841 - val_loss: 0.5006 - val_acc: 0.8929
Epoch 6/20
156/156 [==============================] - 318s 2s/step - loss: 0.0422 - acc: 0.9882 - val_loss: 0.7838 - val_acc: 0.8079
Epoch 7/20
156/156 [==============================] - 318s 2s/step - loss: 0.0388 - acc: 0.9904 - val_loss: 0.8521 - val_acc: 0.8014
Epoch 8/20
156/156 [==============================] - 317s 2s/step - 

In [14]:
vgg16_model = load_model('vgg16_model_15.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=2)])

vgg16_model.save('vgg16_model_11.h5')

Epoch 1/20
156/156 [==============================] - 322s 2s/step - loss: 0.0552 - acc: 0.9832 - val_loss: 0.6105 - val_acc: 0.8594
Epoch 2/20
156/156 [==============================] - 326s 2s/step - loss: 0.0390 - acc: 0.9888 - val_loss: 0.4559 - val_acc: 0.9116
Epoch 3/20
156/156 [==============================] - 325s 2s/step - loss: 0.0243 - acc: 0.9935 - val_loss: 0.5180 - val_acc: 0.8733
Epoch 4/20
156/156 [==============================] - 322s 2s/step - loss: 0.0240 - acc: 0.9933 - val_loss: 1.1330 - val_acc: 0.7099


In [15]:
vgg16_model = load_model('vgg16_model_11.h5')

for layer in vgg16_model.layers[:7]:
    layer.trainable = False
    
for layer in vgg16_model.layers[7:]:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=5e-5),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])
vgg16_model.save('vgg16_model_7.h5')

Epoch 1/20
156/156 [==============================] - 323s 2s/step - loss: 0.0216 - acc: 0.9940 - val_loss: 0.4645 - val_acc: 0.8906
Epoch 2/20
156/156 [==============================] - 327s 2s/step - loss: 0.0190 - acc: 0.9951 - val_loss: 1.9985 - val_acc: 0.5627
Epoch 3/20
156/156 [==============================] - 326s 2s/step - loss: 0.0144 - acc: 0.9963 - val_loss: 0.5850 - val_acc: 0.8645
Epoch 4/20
156/156 [==============================] - 323s 2s/step - loss: 0.0129 - acc: 0.9966 - val_loss: 0.6935 - val_acc: 0.8528


In [16]:
vgg16_model = load_model('vgg16_model_7.h5')

    
for layer in vgg16_model.layers:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-5),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])

vgg16_model.save('vgg16_model_all.h5')

Epoch 1/15
156/156 [==============================] - 323s 2s/step - loss: 0.0058 - acc: 0.9988 - val_loss: 0.3170 - val_acc: 0.9258
Epoch 2/15
156/156 [==============================] - 328s 2s/step - loss: 0.0046 - acc: 0.9990 - val_loss: 0.4538 - val_acc: 0.8802
Epoch 3/15
156/156 [==============================] - 326s 2s/step - loss: 0.0041 - acc: 0.9992 - val_loss: 0.5444 - val_acc: 0.8759
Epoch 4/15
156/156 [==============================] - 324s 2s/step - loss: 0.0041 - acc: 0.9991 - val_loss: 0.4317 - val_acc: 0.8994


In [18]:
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=5e-6),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=3)])

Epoch 1/15
156/156 [==============================] - 324s 2s/step - loss: 0.0028 - acc: 0.9997 - val_loss: 0.3014 - val_acc: 0.9280
Epoch 2/15
156/156 [==============================] - 328s 2s/step - loss: 0.0025 - acc: 0.9996 - val_loss: 0.3899 - val_acc: 0.9046
Epoch 3/15
156/156 [==============================] - 326s 2s/step - loss: 0.0023 - acc: 0.9997 - val_loss: 0.3617 - val_acc: 0.9120
Epoch 4/15
156/156 [==============================] - 322s 2s/step - loss: 0.0019 - acc: 0.9997 - val_loss: 0.4821 - val_acc: 0.8837


In [19]:
vgg16_model.save('vgg16_final_model.h5')

In [20]:
vgg16_model = load_model('vgg16_final_model.h5')

test_model(vgg16_model,test_path,csv='vgg16_submission_0430.csv')